In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -Uqq tsa

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#from tsai.all import *
import math

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, Conv1D, MaxPooling1D

load the preprocessed dataset 

In [5]:
import h5py
hf = h5py.File('/content/drive/MyDrive/Bernaed2/data_int.h5', 'r')
x= hf.get('X')
Y = hf.get('y')

Train test splits

In [12]:
X = np.moveaxis(np.array(x), -1, 1)
#X = np.expand_dims(x, axis=3)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(np.array(X), np.array(Y), test_size=0.25, random_state=42)
X_train.shape,y_train.shape,X_test.shape,y_train.shape

((5052, 100, 126), (5052, 1), (1685, 100, 126), (5052, 1))

Regression models

In [ ]:
model = Sequential()
#model.add(Dense(256, activation='relu', input_dim=366))
model.add(Conv2D(64, (3, 3), activation='relu', input_shape = X_train[0].shape))

model.add(Conv2D(128, (3, 3), activation='relu'))


model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.1))

model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))

model.add(Dense(1, activation='linear'))


model.compile(loss='mean_squared_error', # one may use 'mean_absolute_error' as  mean_squared_error
                  optimizer='adam',
                  metrics=['mse', 'mae', 'mape'] # you can add several if needed
                 )

model.summary()

In [ ]:
model.fit(X_train, y_train, batch_size=64, epochs=100, verbose=1, validation_data=(X_test, y_test) )

In [ ]:
X, y, splits = combine_split_data([X_train, X_test], [y_train, y_test])

In [ ]:
tfms  = [None, [TSRegression()]]

dls = get_ts_dls(X, y, splits=splits, tfms=tfms, bs=128)

In [ ]:
learn = ts_learner(dls, InceptionTime, metrics=[mae, rmse], cbs=ShowGraph())
learn.lr_find()

In [ ]:
learn.fit_one_cycle(10, lr_max=slice(1e-3,1e-1), cbs=ShowGraph())

In [ ]:
learn.fit_one_cycle(10, lr_max=slice(1e-3,1e-1), cbs=ShowGraph())

In [ ]:
learn.lr_find()

In [ ]:
learn.fit_one_cycle(3, lr_max=slice(3e-5,2e-4), cbs=ShowGraph())

In [ ]:
learn2 = ts_learner(dls, InceptionTime, metrics=[mae, rmse], cbs=ShowGraph())
learn2.unfreeze()

In [ ]:
learn2.lr_find()

In [ ]:
learn2.fit_one_cycle(10, lr_max=slice(7e-3,1e-1), cbs=ShowGraph())

In [ ]:
learn3 = ts_learner(dls, XceptionTime, metrics=[mae, rmse], cbs=ShowGraph())
learn3.lr_find()

In [ ]:
learn3.fit_one_cycle(10,  cbs=ShowGraph())

In [ ]:
learn4 = ts_learner(dls, RNN_FCN, metrics=[mae, rmse], cbs=ShowGraph())
learn4.lr_find()


In [ ]:
learn4.fit_one_cycle(10,  lr_max=slice(2e-3,1e-1), cbs=ShowGraph())

In [ ]:
learn.show_results(max_n=9)

In [ ]:
learn.show_results(max_n=9)

In [ ]:
learn.show_results(max_n=9)

In [ ]:
learn5 = ts_learner(dls, ResNet, metrics=[mae, rmse], cbs=ShowGraph())
learn5.lr_find()

In [ ]:
learn5.fit_one_cycle(10,  lr_max=slice(2e-3,1e-1), cbs=ShowGraph())

In [ ]:
tfms  = [None, [TSRegression()]]
batch_tfms = [TSNormalize(),TSToGADF(300)]
dsets1 = TSDatasets(X, y, tfms=tfms, splits=splits)
dls1 = TSDataLoaders.from_dsets(dsets1.train, dsets1.valid, bs=[16,32], batch_tfms=batch_tfms)

In [ ]:
model = create_model(xresnet18, dls=dls1)
learn6 = Learner(dls1, model,  metrics=[mae, rmse])

In [ ]:
learn6.lr_find()

In [ ]:
learn6.fit_one_cycle(10,  lr_max=slice(3e-3,8e-1), cbs=ShowGraph())

In [ ]:
tfms  = [None, [TSRegression()]]
batch_tfms = [TSNormalize(),TSToGADF(400)]
dsets2 = TSDatasets(X, y, tfms=tfms, splits=splits)
dls2 = TSDataLoaders.from_dsets(dsets2.train, dsets2.valid, bs=[16,32], batch_tfms=batch_tfms)

In [ ]:
model7 = create_model(xresnet18, dls=dls2)
learn7 = Learner(dls2, model7,  metrics=[mae, rmse])

In [ ]:
learn7.lr_find()

In [ ]:
learn7.fit_one_cycle(10,  lr_max=slice(3e-3,8e-1), cbs=ShowGraph())

In [ ]:
import scipy
print(scipy.__version__)

1.4.1
